In [49]:
!pip install pymorphy2

In [50]:
import os
import re
from collections import defaultdict
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache
from tqdm import tqdm
import gensim
import pymorphy2
from transformers import MBartTokenizer, MBartForConditionalGeneration
from google.colab import drive
import string
import json
pd.set_option('display.max_columns', 600)

In [51]:
morph = pymorphy2.MorphAnalyzer()

In [52]:
def parse_work(work):
    work_text = f"Дата начала работы: {work['starts']}\n"
    work_text += f"Дата окончания работы: {work['ends']}\n"
    work_text += f"Работодатель: {work['employer']}\n"
    work_text += f"Город: {work['city']}\n"
    work_text += f"Позиция: {work['position']}\n"
    work_text += f"Описание: {work['description']}\n"
    return work_text

def parse_education(education):
    work_text = f"Год обучения: {education['year']}\n"
    work_text += f"Образовательная организация: {education['organization']}\n"
    work_text += f"Обучение: {education['faculty']}\n"
    work_text += f"Направление: {education['specialty']}\n"
    work_text += f"Результат обучения: {education['result']}\n"
    work_text += f"Тип обучения: {education['education_type']}\n"
    work_text += f"Уровень обучения: {education['education_level']}\n"
    return work_text

def parse_resume(resume):
    resume_text = f"ФИО: {resume['first_name']} {resume['last_name']}\n"
    resume_text += f"Дата рождения: {resume['birth_date']}\n"
    resume_text += f"Страна: {resume['country']}\n"
    resume_text += f"Город: {resume['city']}\n"
    if not isinstance(resume['about'], type(None)):
        resume_text += f"Описание: {resume['about']}\n"
    resume_text += f"Ключевые навыки: {resume['key_skills']}\n"
    resume_text += f"Опыт работы:\n"
    if "experienceItem" in resume:
        for idx, work in enumerate(resume["experienceItem"]):
            resume_text += f"Работа номер {idx}\n"
            resume_text += parse_work(work)
    if "educationItem" in resume:
        for idx, education in enumerate(resume["educationItem"]):
            resume_text += f"Обучение номер {idx}\n"
            resume_text += parse_education(education)
    resume_text += f"Владение языками:\n"
    if "languageItems" in resume:
        for language in resume["languageItems"]:
            resume_text += f"{language}\n"
    return resume_text

In [53]:
data_path = "/content/case_2_reference_without_resume_sorted.json"
data_json = [json.load(open(data_path, 'r', encoding='utf-8'))]
dataset_dict = {"id_vacancy":[], "id_resume":[],"description":[], "description_v":[]}
for data in data_json:
    vacancy = data["vacancy"]
    vacancy_id = vacancy["uuid"]
    vacancy_text = f"{vacancy['description']}"
    print(vacancy_text)
    resumes = data["resumes"]
    for resume in resumes:
        dataset_dict["id_vacancy"].append(vacancy_id)
        dataset_dict["id_resume"].append(resume["uuid"])
        if "experienceItem" in resume:
          expirience = ' '.join([exp['description'] if not isinstance(exp['description'], type(None)) else "" for exp in resume['experienceItem']])
        dataset_dict["description"].append(f"{resume['key_skills']} {resume['about'] if not isinstance(resume['about'], type(None)) else ''} {expirience}")
        dataset_dict["description_v"].append(vacancy_text)


Требования: 4+ года опыта работы с Java 8+ или Kotlin 4+ года опыта работы с Spring и 2+ год работы с Spring Boot. Опыт работы с системами на микросервисной архитектуре (Spring Cloud, Kubernetes, Openshift или аналоги). Опыт работы с Docker. Опыт работы с 3-мя любыми из следующих технологий NoSQL (MongoDB, Elasticsearch, аналоги) SQL (PostgreSQL, Oracle, аналоги) Брокеры сообщений (Kafka, RabbitMQ, аналоги) Reactive programming (RxJava, Project Reactor) Cache (Redis, Hazelcast). Настройка CI/CD (GitlabCI, Jenkins, аналоги). Настройка средств мониторинга (Zabbix, Prometheus). Настройка средств логирования (Graylog, ELK). Будет хорошо, если кандидат: Знает все технологии из первого пункта; Участвовал в проработке архитектуры и может объяснить все решения на своем проекте; Готов драйвить техническое развитие систем, а не просто следовать текущим стандартам; Имеет дружеские отношения с DevOps, пайплайны, контейнеризация и оркестрация, Linux; Понимает микросервисную архитектуры и устройство

# Создаём таблицу

In [54]:
# Создаем DataFrame
df = pd.DataFrame(dataset_dict)

# Выводим DataFrame
df

,id_vacancy,id_resume,description,description_v
0,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33,"Java Core, Spring Framework, Hibernate ORM, Po...",Требования: 4+ года опыта работы с Java 8+ или...
1,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,f8b69e24-e2c0-3186-9578-380835eb2ee7,"Java 8-17, Java SE, Java EE, Spring Framework ...",Требования: 4+ года опыта работы с Java 8+ или...
2,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,e3976e74-e71b-34db-8e98-08dc422fa567,"Java 8 и 11, osgi, postgresql, testng, mockito...",Требования: 4+ года опыта работы с Java 8+ или...
3,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,9a9c3ff1-49f8-30dd-a294-e56fc60cae64,"Java EE, Spring Framework, Intellij IDEA, Рабо...",Требования: 4+ года опыта работы с Java 8+ или...
4,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,6561771c-7ef3-3e50-ab3a-ba8547201480,"None О себе: Java/Kotlin-разработчик, знаю Sp...",Требования: 4+ года опыта работы с Java 8+ или...
...,...,...,...,...
108,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,82df355a-235e-3046-9e6e-782ddf1600eb,None BIS. Корпоративная сервисная шина для с...,Требования: 4+ года опыта работы с Java 8+ или...
109,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,915597ce-24e5-31fa-8dca-29437f49f839,"Java Servlets, Hibernate ORM, SOAP, REST, Spri...",Требования: 4+ года опыта работы с Java 8+ или...
110,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,f288a532-0b58-30cb-ac3c-f87e53984719,"Ответственность, Коммуникабельность, Аналитиче...",Требования: 4+ года опыта работы с Java 8+ или...
111,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,3e3a379f-226e-305e-b7d8-cf341e00cbd7,"Linux, Apache Maven, Java Collections, Java EE...",Требования: 4+ года опыта работы с Java 8+ или...


In [55]:
vacancies = set(df.id_vacancy.tolist())
vacancies = list(vacancies)
vacancies

['8b9c8d16-c7f0-38a2-b80c-d94030c15a6f']

In [56]:
data = df.loc[df['id_vacancy']==vacancies[0]]
data

,id_vacancy,id_resume,description,description_v
0,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,0dfe8e63-d7a3-3fe4-b9d7-1b8122158f33,"Java Core, Spring Framework, Hibernate ORM, Po...",Требования: 4+ года опыта работы с Java 8+ или...
1,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,f8b69e24-e2c0-3186-9578-380835eb2ee7,"Java 8-17, Java SE, Java EE, Spring Framework ...",Требования: 4+ года опыта работы с Java 8+ или...
2,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,e3976e74-e71b-34db-8e98-08dc422fa567,"Java 8 и 11, osgi, postgresql, testng, mockito...",Требования: 4+ года опыта работы с Java 8+ или...
3,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,9a9c3ff1-49f8-30dd-a294-e56fc60cae64,"Java EE, Spring Framework, Intellij IDEA, Рабо...",Требования: 4+ года опыта работы с Java 8+ или...
4,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,6561771c-7ef3-3e50-ab3a-ba8547201480,"None О себе: Java/Kotlin-разработчик, знаю Sp...",Требования: 4+ года опыта работы с Java 8+ или...
...,...,...,...,...
108,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,82df355a-235e-3046-9e6e-782ddf1600eb,None BIS. Корпоративная сервисная шина для с...,Требования: 4+ года опыта работы с Java 8+ или...
109,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,915597ce-24e5-31fa-8dca-29437f49f839,"Java Servlets, Hibernate ORM, SOAP, REST, Spri...",Требования: 4+ года опыта работы с Java 8+ или...
110,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,f288a532-0b58-30cb-ac3c-f87e53984719,"Ответственность, Коммуникабельность, Аналитиче...",Требования: 4+ года опыта работы с Java 8+ или...
111,8b9c8d16-c7f0-38a2-b80c-d94030c15a6f,3e3a379f-226e-305e-b7d8-cf341e00cbd7,"Linux, Apache Maven, Java Collections, Java EE...",Требования: 4+ года опыта работы с Java 8+ или...


In [57]:
with open('stopwords-ru.txt', 'r') as f:
    stop_words = f.read().split('\n')
len(stop_words)

559

In [58]:
stop_words_ = """дабы, лишь только, таким, для, на, по, со, из, от, до, без, над, под, за, при, после, во, же, то, бы, всего, итого, даже, да, нет, ой, ого, эх, браво, здравствуйте, спасибо, извините, скажем, может, допустим, честно говоря, например, на самом деле, однако, вообще, в, общем, вероятно, очень, минимально, максимально, абсолютно, огромный, предельно, сильно, слабо, самый, сайт, давать, всегда, однако, и, а, но, да, если, что, когда, потому, что, так, как, как, будто, вследствие, того, что, с, тех, пор, как, в, то, время, как, для, того, чтобы, ни, то, ли, но, зато, от, и, к, система, сотрудник, компания, проект, стек"""
len(stop_words_)

620

In [59]:
stop_words_1  = set(stop_words + stop_words_.replace('\n','').split(', '))
print(stop_words_1)

{'могут', 'именно', 'народ', 'пожалуйста', 'общем', 'могу', 'минута', 'тем', 'ой', 'на самом деле', 'понять', 'бывает', 'восемнадцать', 'разве', 'эх', 'хороший', 'раз', 'своих', 'часто', 'ничто', 'люди', 'наша', 'сказать', 'конец', 'потом', 'те', 'отец', 'мимо', 'ого', 'вон', 'твои', 'лишь', 'хорошо', 'конечно', 'любить', 'случай', 'её', 'нужный', 'ниже', 'итого', 'зато', 'впрочем', 'самому', 'они', 'стол', 'времени', 'двух', 'четыре', 'самый', 'предельно', 'новый', 'человек', 'та', 'машина', 'всею', 'свет', 'страна', 'лежать', 'время', 'может', 'всё', 'десять', 'этими', 'тех', 'хотя', 'лет', 'ещё', 'назад', 'чаще', 'десятый', 'ней', 'дальше', 'видеть', 'говорил', 'даже', 'которого', 'за', 'шестнадцать', 'дать', 'хотел бы', 'одной', 'них', 'занят', 'и', 'буду', 'таким', 'здравствуйте', 'вообще', 'казаться', 'вверх', 'кажется', 'слишком', 'были', 'перед', 'неё', 'вдруг', 'шесть', 'сказала', 'лицо', 'восьмой', 'немного', 'помнить', 'нее', 'однажды', 'однако', 'война', 'мои', 'кем', 'плеч

In [60]:
len(stop_words_1)

590

In [61]:
for i in range (len(vacancies)):
  data = df.loc[df['id_vacancy'] == vacancies[i]]
  texts = []
  content_values = data['description'].tolist()
  for value in content_values:
    texts.append(value)
  def clear_text(t):
    t = str(t).lower()
    t = t.replace('\n', '  ')
    t = t.replace('.', '. ')
    t = t.replace(',', ', ')
    t = t.replace('ха0', ' ')
    return ' '.join(re.findall('[a-za-яё]+', t))
  texts_cleared = []
  for text in texts:
    texts_cleared.append(clear_text(text))
  texts_tokenized = [t.split() for t in texts_cleared]
  texts_tokenized[-len(texts_tokenized)]
  @lru_cache(100000)
  def lemmatize(s):
    s = str(s).lower()
    return morph.parse(s)[0].normal_form
  texts_tokenized_1 = [[lemmatize(tt) for tt in t if len(tt) > 1]
                     for t in tqdm(texts_tokenized)]
  texts_tokenized_1 = [[tt for tt in t if tt not in stop_words_1]
                     for t in texts_tokenized_1]
  texts_tokenized_1[-len(texts_tokenized_1)]
  ttfs = defaultdict(int)
  dfs = defaultdict(int)
  for t in texts_tokenized_1:
    for term in set(t):
        if len(term) > 1:
            ttfs[term] += t.count(term)
            dfs[term] += 1
  df_dfs = pd.DataFrame({'cnt_dfs': pd.Series(dfs), 'cnt_ttfs': pd.Series(ttfs)})
  df_dfs.index.name = 'term'
  df_dfs = df_dfs.reset_index()
  df_dfs = df_dfs.sort_values('cnt_ttfs', ascending=False)
  df_dfs_1 = df_dfs[(df_dfs['cnt_ttfs'] > 1)]
  dfs_1 = df_dfs_1.to_dict('records')
  dfs_1 = {i['term']: i['cnt_dfs'] for i in dfs_1}
  terms = set(dfs_1.keys())
  texts_tokenized_2 = [[tt for tt in t if tt in terms] for t in texts_tokenized_1]
  #%%time

  model = gensim.models.FastText(texts_tokenized_2, min_count=1, negative=5,
                               vector_size=100, window=5, workers=16)
  vars(model.wv)
  model.wv.vectors_vocab = None
  model.wv.vectors = model.wv.vectors.astype(np.float16)
  model.wv.vectors_ngrams = model.wv.vectors_ngrams.astype(np.float16)

  def text_vec(t):
    res = np.zeros(100)
    cnt = 0
    for tt in t[:60]:
        try:
            res += model.wv[tt]
            cnt += 1
        except:
            pass
    if cnt > 0:
        return res / (cnt * np.linalg.norm(res / cnt))
    else:
        return res
  np.linalg.norm(text_vec(texts_tokenized_2[0]))
  texts_mean_vect = [text_vec(t) for t in tqdm(texts_tokenized_2)]
  np_texts_mean_vect = np.vstack(texts_mean_vect)

  #query = 'Описание Мы расширяем команды и ищем разработчиков для развития нескольких сервисов:   Инвестиции. Мы — лидер среди брокеров по количеству активных клиентов. Делаем инвестиции удобными, технологичными и понятными;   Бизнес. Меняем подход к ведению бухгалтерии, обмену документами между организациями и работе с архивами — переводим все и вся в цифру;   Страхование. Развиваем платформу прямых продаж. Наша цель — предсказуемый, удобный процесс для бизнеса и клиента; Платежные технологии и процессинг. Занимаемся разработкой и поддержкой платежного шлюза банка. Задачи шлюза — определять тип платежей, наполнять их данными из других систем банка и проверять разрешенность операций. У нас много интересных и разнообразных задач, опытная команда и отличные возможности для роста. Откликайтесь на вакансию, чтобы узнать о проектах и выбрать подходящий для вас. Требования Опыт разработки на Java от 3 лет Опыт коммерческой разработки на Java 11+ или Kotlin Опыт коммерческой разработки с любым из фреймворков: Spring Boot, Quarkus, Micronaut или Vert.x Опыт коммерческой разработки с одним из контейнеризаторов: Kubernetes, Docker или OpenShift Опыт коммерческой разработки с одним из брокеров: Kafka, Rabbit MQ или Active MQ Опыт коммерческой разработки с Postgress, MySQL или Oracle будет плюсом Опыт работы с системой контроля версий Мы предлагаем Работу в офисе или удаленно — по договоренности Возможность работы в аккредитованной ИТ-компании Платформу обучения и развития «  Апгрейд». Курсы, тренинги, вебинары и базы знаний. Поддержку менторов и наставников, помощь в поиске точек роста и карьерном развитии Заботу о здоровье. Оформим полис ДМС со стоматологией и страховку от несчастных случаев. Предложим льготное страхование вашим близким Бесплатный фитнес-зал или компенсацию затрат на спортивные занятия 3 дополнительных дня отпуска в год Уникальную well-being-программу, направленную на физическое и ментальное благополучие сотрудников Достойную зарплату — обсудим ее на собеседовании'
  query = data['description_v']
  query = clear_text(query)
  vect_query = text_vec(query.split()).reshape(-1,1) / np.linalg.norm(text_vec(query.split()))
  np.linalg.norm(vect_query)

  relevance = np.matmul(np_texts_mean_vect, vect_query)
  top_ind = (-relevance.flatten()).argsort()
  print(top_ind)
  print(relevance[top_ind])


100%|██████████| 113/113 [00:00<00:00, 705.06it/s]


[ 81 105  41   5  37   2  42  36  70   9  12 104  17  32  23  83 107  48
   0 110  25  51   3  75  56  61  62   6 109  46  67  33  40  96  91  45
 103  72  86  69  66  64   7  30  53  92 111 106  82  80  11  21  93  10
  76  47  89  20  35  44  16  73  34 108  38  77  50  95  24   8  19  54
  15  55  14  97  13  18  79  58  31  27  87  98  52  88  60  39  99  68
 112 102  65  94  28 100  63   1  78  57  26  85  49  22 101  71  74  59
  29  43  84  90   4]
[[0.9999871 ]
 [0.99998678]
 [0.9999865 ]
 [0.99998637]
 [0.99998617]
 [0.999986  ]
 [0.99998582]
 [0.99998579]
 [0.99998549]
 [0.99998504]
 [0.99998501]
 [0.99998501]
 [0.99998485]
 [0.99998477]
 [0.99998469]
 [0.99998462]
 [0.99998369]
 [0.99998278]
 [0.99998263]
 [0.99998252]
 [0.99998246]
 [0.99998175]
 [0.99998117]
 [0.9999809 ]
 [0.99997987]
 [0.99997953]
 [0.99997947]
 [0.99997878]
 [0.99997869]
 [0.99997796]
 [0.99997784]
 [0.99997765]
 [0.99997546]
 [0.99997538]
 [0.99997509]
 [0.99997505]
 [0.9999744 ]
 [0.99997434]
 [0.9999

In [62]:
import csv
for i in top_ind[:10]:
  print(df["id_resume"][i])
with open("ans.csv", "w",newline='',encoding="utf-8") as f:
  writer = csv.writer(f)
  field = ["resume_uuid"]
  writer.writerow(field)
  for i in top_ind[:10]:
    writer.writerow([str(df["id_resume"][i])])

da6466e6-0b6d-36aa-adbf-9f0e4fdd6b3f
0f45f507-0b5d-347f-86e2-0f799898f812
8b1dc5d0-dde1-31be-851f-a643ae235d50
5ae3b7a0-6028-3bd8-8b4e-27a861503806
63b33138-5045-31fc-878b-65672a829658
e3976e74-e71b-34db-8e98-08dc422fa567
639c1eda-c4ec-3319-b2e2-17ad5d87e173
b1357f8e-3c98-3142-81e7-fd729ec50ca0
5c8ce3e9-f9dd-3ed2-b044-d55c2957937e
5785c202-6744-3e1b-994a-d5bffc6aad14
